#### High level Agent Executor with langgraph

In this section we will be looking at a high level implementation of the agent executor in langgraph. We will be abstracting a lot of the concepts away, and only be focusing on wrappers at a high level

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tools = [TavilySearchResults(max_results=1)]

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")
agent_runnable = create_openai_functions_agent(llm, tools=tools, prompt=prompt)

##### Create Agent Executor
Now we are going to use highlevel apis provided by langchain to define our agent executor graph

In [4]:
from langgraph.prebuilt import create_agent_executor
app = create_agent_executor(agent_runnable, tools)

In [6]:
inputs = {
    "input": "what is the weather in san fransisco",
    "chat_history": []
}

for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentFinish(return_values={'output': ''}, log='')}
----
{'input': 'what is the weather in san fransisco', 'chat_history': [], 'agent_outcome': AgentFinish(return_values={'output': ''}, log=''), 'intermediate_steps': []}
----


In [7]:
s["__end__"]["agent_outcome"]

AgentFinish(return_values={'output': ''}, log='')

#### Custom Input Schema
By default, the `create_agent_executor` assumes that the input will be a dictionary with two keys: `input` and `chat_history`. If this is not the case, you can easity customize the input schema. You should do this, by deifning a schema as a TypedDict

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Respond to the user qeustion: {question}. Answer in this language {language}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [9]:
from typing import TypedDict

class InputSchema(TypedDict):
    question: str
    language: str

In [11]:
app = create_agent_executor(agent_runnable, tools, InputSchema)

In [12]:
inputs = {"question": "what is the weather in sf", "language": "italian"}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentFinish(return_values={'output': ''}, log='')}
----
{'question': 'what is the weather in sf', 'language': 'italian', 'agent_outcome': AgentFinish(return_values={'output': ''}, log=''), 'intermediate_steps': []}
----
